In [1]:
import pandas as pd
import numpy as np
import multiprocessing
from bs4 import BeautifulSoup as bs
from selectolax.parser import HTMLParser
import re
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import string

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from time import time  # To time our operations
from collections import defaultdict, Counter  # For word frequency

# import logging  # Setting up the loggings to monitor gensim
# logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/adwiz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/adwiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Data loading and preparation
data = pd.read_json('../datasets/dataset.json')
mapping = {False: 0, True: 1}
data.replace({'hasBadWords': mapping}, inplace=True)
# data.hasBadWords = data.hasBadWords.apply(lambda x: 1 if x == True else 0)
data.rename(columns={"hasBadWords": "labels"}, inplace=True)
data.drop(['violation'], axis=1, inplace=True)
data.shape

(86439, 2)

In [4]:
data.head()

,text,labels
0,My Favorite Slut,0
1,girlfriends sit on each other's faces with the...,0
2,bound beauty kisses her girlfriend,0
3,MORGAN - Anytime - Nail Painting On The Slave'...,0
4,TRANSGENDER COACHING (wmv) PART 1,0


# Preprocessing function

In [5]:
def preprocess(text, stop_words, punctuation_marks): #, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = token #morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return ' '.join(preprocessed_text)

punctuation_marks = ['!', ',', ';', '(', ')', ':', '-', '--', '?', '@', '....', '~',
                     '.', '..', '...', '<', '>', '=', '»', '|', '’', '`', '+', '$',
                     '&', '#',  '*', '``', '%', '[', ']', '{', '}']

stop_words = stopwords.words('english') + ['14000kbps', '1080p', "n't", "'s", '4k', "'m", 'mp4', 'error', '404']
morph = pymorphy2.MorphAnalyzer(lang='uk')

In [6]:
def normalize(s):
    """
    Given a text, cleans and normalizes it. Feel free to add your own stuff.
    """
    s = s.lower()
    # Replace ips
    s = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', ' _ip_ ', s)
    # Isolate punctuation
    s = re.sub(r'([.\(\)\!\?\-\\\/\,])', r' \1 ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Replace numbers and symbols with language
    s = s.replace('&', ' and ')
    s = s.replace('@', ' at ')
    s = s.replace('0', ' zero ')
    s = s.replace('1', ' one ')
    s = s.replace('2', ' two ')
    s = s.replace('3', ' three ')
    s = s.replace('4', ' four ')
    s = s.replace('5', ' five ')
    s = s.replace('6', ' six ')
    s = s.replace('7', ' seven ')
    s = s.replace('8', ' eight ')
    s = s.replace('9', ' nine ')
    return s

In [7]:
data[:1000].apply(lambda row: str(row.text), axis=1)

0                                       My Favorite Slut
1      girlfriends sit on each other's faces with the...
2                     bound beauty kisses her girlfriend
3      MORGAN - Anytime - Nail Painting On The Slave'...
4                      TRANSGENDER COACHING (wmv) PART 1
                             ...                        
995    Captain Next Fucks Hot Wife Laney On Her Husba...
996    Captain Next Fucks Hot Wife Laney On Her Husba...
997    Captain Next Fucks Hot Wife Laney On Her Husba...
998    XY19 - Sexy Chinese Maid Gets Femdom With Vibr...
999     The Strongest Reverse Headscissors Contest HDMP4
Length: 1000, dtype: object

In [8]:
# data[:100].apply(lambda row: bs(row['text']).get_text().replace('\n',' '),axis=1)

In [9]:
data[:100].apply(lambda row: preprocess(row.text, punctuation_marks, stop_words), axis=1) #, morph), axis=1)

0                                         favorite slut
1                           girlfriends sit faces asses
2                        bound beauty kisses girlfriend
3               morgan anytime nail painting slave face
4                       transgender coaching wmv part 1
                            ...                        
95    real mesmerized housewife brooklyn follows orders
96                     kaylia strict hogtie naked floor
97                 real mesmerized submissive housewife
98                        ehesklave muss füsse lutschen
99    uncut buck nekkid fucks creampies polly bundy ...
Length: 100, dtype: object

In [10]:
#data['text_preprocessed'] = data.apply(lambda row: row.text.replace(r'\n', '').replace(r'\r', ' '), axis=1)
data['text_preprocessed'] = data.apply(lambda row: bs(row.text).get_text().replace(r'\n', '').replace(r'\r', ' ').replace(r',', ' '), axis=1)
# data['text_preprocessed'] = data.apply(lambda row: HTMLParser(row.text).body.text(separator=' ').replace('\n',' '),axis=1)

In [ ]:
# s = "foot,feet,toe,sole,footjob,toejob,sockjob,nylons,fetish,foot fetish,toes,soles,arch,footjobs,toejobs,amateur,oral,anal,hardcore,cumshot,polished,unpolished,handjobs,oily,handjob,cum,tickling,bondage,kinky,boots,video,clip,dvd,vhs,credit card,vendor,low price,custom,vaseline,tip,wet,homemade,blowjob,blowjobs,tits,titjob,boobs,shaved,pussy,young,foot,feet,toe,sole,footjob,toejob,sockjob,nylons,fetish,foot fetish,toes,soles,arch,footjobs,toejobs,amateur,oral,anal,hardcore,cumshot,polished,unpolished,handjobs,oily,handjob,cum,tickling,bondage,kinky,boots,video,clip,dvd,vhs,credit card,vendor,low price,custom,vaseline,tip,wet,homemade,blowjob,blowjobs,tits,titjob,boobs,shaved,pussy,young,foot,feet,toe,sole,footjob,toejob,sockjob,nylons,fetish,foot fetish,toes,soles,arch,footjobs,toejobs,amateur,oral,anal,hardcore,cumshot,polished,unpolished,handjobs,oily,handjob,cum,tickling,bondage,kinky,boots,video,clip,dvd,vhs,credit card,vendor,low price,custom,vaseline,tip,wet,homemade,blowjob,blowjobs,tits,titjob,boobs,shaved,pussy,young,tease,shoe,sandal,nylon,sandaljob,shoejob,asian,raw,store,footjob,toes,soles,cum,cock,footjob,foot,jobs,soles,toes,foot,fetish,\r\nfoot,worship,solesquirters,cum,shot,masturbation,mature,\r\nsoles,worship,bitches-feet,worship,feet licking,femdom,mistress,\r\nslave,huminilation,facesitting,sexy,footjobs,stocking,\r\nworship,cock,soles,trampling,pinching,DARLA,FEET,WIFE,alicia,fucking,interracial,  \r\nct,amature,cam,caught,fucking,in,sex,str8,ass,bareback,bb,\r\nbreed,cum,inside,raw,bed,boner,cock,guy,penis,sweet,china,\r\ndan,singapore,stud,my,room,finger,the,call,can,making,me,\r\nmonster,well,fetish,gay,mouth,piss,black,white,amateur,amature,male,\r\nsperm,ball,ballbuster,ballbusting,balls,busted,busting,busts,foot,\r\ngirl,hot,kick,kicked,kicks,nuts,pink,shoes,sneakers,erotic,\r\nhose,knee,nut,painful,panty,pantyhose,testicle,testicles,buster,\r\ndefense,demonstration,hit,interview,interviewed,self,18,babe,bare,\r\nbarefoot,beach,bikini,blonde,feet,kneed,to,bitch,bitches,blond,\r\ngroin,kneeling,back,girls,heel,hurt,pain,ballbusters,boot,boots,heels,\r\nsandals,sarah,black,redhead,asian,yellow,green,blue,red,polished,unpolished,long,\r\ntoenail,long feet,large,\r\nblonde,blowjob,cumshot,facial,fucking,ipod,psp,\r\nbang,bondage,gang,mask,men,orgy,redhead,sitter,3-way,3way,\r\nanal,brunette,condom,cum,fuck,jizz,load,huge,shoot,suck,tattoo,\r\narmpit,core,fetish,katrena,panty , porn , pornstar , soft , softcore ,\r\nsolo , star , starr , tall , thong , brazil , oral , sex , sexo , girl ,\r\npussy , bicep , cock , dick , face , lick , nipple , penis , swallow ,\r\ntaste , twink , uncut , hot , jerk , off , spunk , wank , gay , gratefuldead,\r\nhung , me , naked , nude , toe , buzzed , young , fist , fisted , focus , \r\norgasm , toys , dildo , first , boobs , dyke , girls , lesbo , lesbos ,\r\nlovers , sexy , breast , hottie , fisting , amateur , bj , job , tit , tits ,\r\nanus , asshole , public , subway , toying , deep , outdoors , throat , vintage, \r\nbig , black , huge , lesbians , strapon , erotic , erotica , model , modeling ,\r\nnatural , penthouse , bikini , shower , wet , dont_give_a_fuck,  \r\nasian , gay , jack , off , ladyboy , pantyhose , shemale , transsexual ,\r\nboys , cum , spunk , wank , guy , muscle , guys , latino , men , sex ,\r\nsuck , big , girls , have , tits , amateur , cock , nude , anal , blowjob ,\r\nindian , hunks , interracial , gangbang , gonzo , midget , oriental , ass ,\r\nbubble , masturbate , trannsexual , trannssexual , tranny , transexual ,\r\ntransvestite , ts , tv , japan , japanese , jo , oral , angel , athelete, \r\ntwinkle , asia , zen , beach , hairy , straight , boy , cute , hunk , korean ,\r\nmasturbation , fuck , twink , butt , filipino , jerkoff , load , 18cm , china, \r\nchinese , hk , hong , jerk , kong , bigtits , boobs , busty , cumshot , hardcore,\r\nmanga , pussy , toon , bukkake , jerk-off , orgy , 0524 , bulacan , burat ,\r\nceu , chupa , hogonoy , malolos , manila , marcelo , pampanga , philippines ,\r\npinoy , scanda , tamod , tsupa , dance , dancing , naked , shake , shaking , \r\nlovers , scandal , my , jockstrap , outside , stripping , night , vision ,\r\njungle , twink. , twinks ,  , dirty,\r\namateur , fuck , twink , action , gay , model , big , cock , cum , jerk-off ,\r\nload , man , sperm , wank , webcam , gonzo , boyfriend , dicks , me ,\r\nmy , privat , boobs , tits , anal , step-daddy , dildo , fetish , older , jo ,\r\noutdoor , glory , hole , quicky , bareback , group , orgy , sex , brunette ,\r\ncumshot , dick , pounded , sexy , toys , masturbation , str8-bait , nude ,\r\ncollege , raw , studs , twinks , blowjob , bigcock , cute , huge , jack , jerking,\r\nmasturbate , off , solo , tease , thick , wanking , asian , ejaculation , hard,\r\npre-cum , semen , straight , vagina , penis , 18 , young , jizz ,\r\nmasturbation , spunk , uncut , shoot , cumming , male , ass , butt , naked ,\r\njerks-off , masturbatin , wanks , 18-21 , cam , frat , jerk , skate , skater ,\r\nsurfer , eat , \r\nanal , ass , bi , bisexual , boner , boy , cum , frat , gay , group , hard, \r\nhunk , jock , muscle , pussy , straight , bareback , breeding , cock , ebony,\r\ninterracial , raw , fuck , self , video , hardcore , sex , dick , fucking ,\r\npenis , swing , swinging-free , double , penetration ,blow , step-brother,\r\nbrothers , gangbang , groupsex , hot , male , stud , ball , bizarre , butt,\r\ncumming , dildo , fisting , hand , play , chulito , cogida , elsa , elza ,\r\ngratis , porno , yummi , rough , smooth , amateur , big , step-daddy , fetish,\r\nolder , sperm , 3-way , 3way , blonde , brunette , condom , jizz , load, \r\nshoot , suck , tattoo , black , job , big-tits , gonzo , all-sex , m.i.l.f. ,\r\nsquirting , cream-pies , cumshot , threeway , blowjob , hentai , manga , toon ,\r\nbusty , doggystyle , gag , selfcum , slime , glass , masturbate , solo ,\r\nwebcam , latino , latin , spanish-speaking , bear , man , asian , cartoon ,\r\nfucked , college , studs , twinks , machine , piss , erect , film , french,\r\ngorgeous , men , movie , balls , deep,  \r\namature , balls , cbt , cock , dick , fetish , hardcore , jack , jo , nasty,\r\noff , penis , pig , play , pnp , prick , stroke , wank , act , cam , caught,\r\nfucking , in , sex , str8 , chaps , cockring , exhibitionist , flutiesteak ,\r\njackoff , jerkoff , leather , masterbate , wifebeater , gay , male , nude ,\r\ndenim , exhibitionism , ripped , shorts , me , outdoors , self , bulge ,\r\nnylon , public , asian , fuck , cum , orgasm , fluff , hard , cocks, \r\ncumming , general , masturbating , big , jacking , jock , stud , young,\r\nebony , interracial , outdoor , bi , glory , gloryhole , hole , oral,\r\nsucking , fingering , homemade , video , amateur , ass , butt , friend,\r\nkinky , panties , panty , solo , straight , briefs , condom , hairy , load,\r\nuncut , underwear , belly , black , breasts , lingerie , pussy , rubbing,\r\ntouch , hand , anal , build , handsome , hunks , japanese , moarning , nice ,\r\nspandex , feet , face , hot , 18 , bra , hung , jerk , jerking , lad ,\r\nshemale ,tranny , dildo , gapped , guy,  \r\ncum , down , nylon , jacket , coat , parka , cock , polyamide , shiny, \r\nslick , smooth , quilted , filthy , come , insulated , dirty , stained,\r\nnuptse , ortalion , padded , fetish , orange , bomber , vest , helmet,\r\n ,gloves , mask , ridingboot , zodiac , chestwaders , rubber , pvc , latex ,\r\nwanking , cumming , jockstrap , faggot , moobs , pudgy , suited , suit ,\r\nsilk , satin , masterbate , living , helly , hansen , charlee , chase ,\r\nmilf , step-mom , pornstar , porn , puffy , suck , sucking , pussy , fuck ,\r\nfucking , mesh , muscle , shorts , shinnyboy , penis , jizz , hunk ,\r\nbig , dick , huge , sperm , ground , baseball , tnf , spunk , perv ,\r\n  , thief , puffa , fucker , trench , chubby , trackies , polyester ,\r\n18 , 19 , 17 , jackets , pants , snow , sports , hood , breath , sho ,\r\nhikaru , west , japanese , asian , snorkel , jackoff , fur , winter ,\r\nmasturbation , apparel , strap , pose , mac , raincoat , rain , jerk-off ,\r\nwank , jacking , gay , hard , hardcore , hung , men , oral , rimming , sex ,\r\nshoot,cum , jerk-off , masturbation , wank , jacking , cock , fuck , fucking ,\r\ngay , hard , hardcore , hung , men , oral , rimming , sex , shoot ,\r\nshooting , anal , ass , bi , bisexual , boner , boy , frat , group ,\r\nhunk , jock , muscle , pussy , straight , toss , twink , ebony , horny ,\r\njerk , jizz , office , selfsuck , solo , feet , fetish , foot , sleepy ,\r\nbathroom , big , bikini , hot , jo , self , 19 , cums , young , \r\njack , masturbate , off , sperm , str8 , amateur , load , man , webcam ,\r\ndick , erection , hardon , jerking , orgasm , penis , spunk , wanking , \r\njackoff , porn , 18 , cumshot , nude , sweet , y/o , yo , dirty , orgy ,\r\nsnowball , sucks , dude , j/o , piss , huge , naked , camo , balls ,\r\nmasterbation , smooth , twinks , blow , step-brother , brothers , gangbang ,\r\ngroupsex , outdoor , college , mixed , moaning , facial , jerkoff , small,\r\nuncut , shot , chest , crossdress , satin , transvestite , tv , underwear ,\r\nchained , cushion , homemade , sponge , black , rough , cuffs , \r\ncock , hairy , piss , shorts , smoking , cum , fuck , fucking , gay , hard,\r\nhardcore , hung , men , oral , rimming , sex , shoot , shooting , ebony,\r\nhorny , jerk , jizz , office , cam , male , muscle , pose , big , black, \r\ndick , bathroom , bikini , hot , hunk , jo , self , anal , bareback , breeding,\r\ninterracial , raw , amateur , jerk-off , load , man , sperm , wank , webcam, \r\nboner , erection , hardon , jacking , jerking , masturbation , orgasm , penis ,\r\nspunk , wanking , jackoff , porn , huge , bed , dildo , naked , bear , belly ,\r\nchub , chubby , fat , off , ass , shower , abercrombie , bi , bisexual ,\r\n frat , jock , muscles , muscular , stud , balls , exhibitionisme , masturbate,\r\n , stroking , fucked , nipple , sweaty , tattoo , cumshot , cut , jerkoff ,\r\n thick , asshole , boy , dirty , fart , finger , fingering , hole , manhole ,\r\n, nice ,  ,  , facial , chest , yummi , rough , smooth , cuffs , \r\nhand , pecker , soft , boyfriend , dicks , me , my , private , masterbate ,\r\nandre , dre ,smothering,worship,shoes,shoejob,sandals,cock,cumshot,cum,shoes,shoejob,\r\nsandals,cock,shoejob,goddess,female,humiliation,bitchs,lave,dreamgirls,socks,\r\nhi,heels,licking,snif,face,sit,sitting,girls,girl,roomate,panties,panty,ass,butt,\r\nbum,sub,masorotica,FJ,HJ,Oil,Lube,blonde,nylon,lesbians,breanna,productions,\r\nboot,boots,lick,licking,financial,domination,candid,finger,hand,trample,couch,ring,\r\nasian,spreading"
# print(bs(s).get_text().replace('\r', '').replace('\n', ' '))

In [11]:
data['text_preprocessed'] = data.apply(lambda row: preprocess(row.text_preprocessed, punctuation_marks, stop_words), axis=1)

In [12]:
# Dropping the rows with "Sci"
# identify partial string
discard = ["<div ", "<p ", "<span ", "<p>", "<div>", "<h", "<input ", "center>", "<a ", 
           "<td>", "<", ">", "            ", "Ø"]
  
# # drop rows that contain the partial string "dev"
# data[~data.text.str.contains('|'.join(discard))]

data = data[~data.text.str.contains('|'.join(discard))]#data[~data['text'].str.contains('<div' or '<p>' or '<b'or '<br>' or '&nbsp' or '=')==False]

In [13]:
data

,text,labels,text_preprocessed
0,My Favorite Slut,0,favorite slut
1,girlfriends sit on each other's faces with the...,0,girlfriends sit faces asses
2,bound beauty kisses her girlfriend,0,bound beauty kisses girlfriend
3,MORGAN - Anytime - Nail Painting On The Slave'...,0,morgan anytime nail painting slave face
4,TRANSGENDER COACHING (wmv) PART 1,0,transgender coaching wmv part 1
...,...,...,...
83594,"ebony,hotwife,wife,swinger,cuckold,bigass",0,ebony hotwife wife swinger cuckold bigass
83595,"ssbhm, bhm, ffa, female fat admire, fat admire...",0,ssbhm bhm ffa female fat admire fat admire fee...
83596,"Feet in heels, sexy shoes, high heels, high he...",0,feet heels sexy shoes high heels high heel fet...
83597,"foot fetish, breeding, kinky, fetish porn, bon...",0,foot fetish breeding kinky fetish porn bondage


In [14]:
data = data[['text_preprocessed', 'labels', 'text']] # columns reorder

In [15]:
data.head()

,text_preprocessed,labels,text
0,favorite slut,0,My Favorite Slut
1,girlfriends sit faces asses,0,girlfriends sit on each other's faces with the...
2,bound beauty kisses girlfriend,0,bound beauty kisses her girlfriend
3,morgan anytime nail painting slave face,0,MORGAN - Anytime - Nail Painting On The Slave'...
4,transgender coaching wmv part 1,0,TRANSGENDER COACHING (wmv) PART 1


In [ ]:
data.to_csv('../datasets/clear_text.csv')

# Tensorflow

In [ ]:
# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count  

In [ ]:
counter = counter_word(data.text_preprocessed)  

In [ ]:
data.text_preprocessed

In [ ]:
num_unique_words = len(counter)

In [ ]:
num_unique_words

In [ ]:
counter

In [ ]:
counter.most_common(150)

In [ ]:
from wordcloud import *
word_freq = [i for i in counter.most_common(50)]
wd = WordCloud(background_color='white')
wd.generate_from_frequencies(dict(word_freq))
plt.figure()
plt.imshow(wd, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
train_size = int(data.shape[0] * .8)

train = data[:train_size]
val = data[train_size:]

# Split train and test

X_train = train.text_preprocessed.to_numpy()
y_train = train.labels.to_numpy()

X_val = val.text_preprocessed.to_numpy()
y_val = val.labels.to_numpy()

In [ ]:
train.text_preprocessed.to_numpy()

In [ ]:
X_train,  y_train

In [ ]:
X_train.shape, X_val.shape

# Tokenize

In [ ]:
# vectorize a text corpus by turning each text into sentence of integers

tokenizer2 = Tokenizer(num_words=num_unique_words)
tokenizer2.fit_on_texts(X_train)

In [ ]:
import pickle

# saving
with open('../models/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer2, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('../models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
# each word have a unique index
word_index = tokenizer.word_index
word_index

In [ ]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_val_sequences = tokenizer.texts_to_sequences(X_val)

In [ ]:
print(X_train[1])
print(X_train_sequences[1])

In [ ]:
# pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# max words in a sequence
max_length = 20

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding="post", truncating="post")
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_length, padding="post", truncating="post")

X_train_padded.shape, X_val_padded.shape


In [ ]:
X_train_padded[10]

In [ ]:
print(X_train[10])
print(X_train_sequences[10])
print(X_train_padded[10])

In [ ]:
# Check reversing the indices

# flip (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [ ]:
reverse_word_index

In [ ]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

decoded_text = decode(X_train_sequences[10])

print(X_train_sequences[10])
print(decoded_text)

In [ ]:
# Create LSTM model
from tensorflow.keras import layers

# Word embeddings give us a way to use an efficient, dense representation in which similar words have
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify)
# tf.debugging.set_log_device_placement(True)

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

# The layer will take as input as integer matrix of size (batcg, input_length)
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.

model.add(layers.LSTM(64, dropout=.1))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
from pathlib import Path
path = Path('../models/bad_words.model')
path.mkdir(exist_ok=True) 
cpt_filename = '{epoch:02d}_checkpoint_{val_loss:.2f}.hdf5'
cpt_path = str(path / cpt_filename)

checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_path, monitor='val_loss', verbose=1, 
                                                save_best_only=True, mode='min')

In [ ]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optimizer = keras.optimizers.Adam(learning_rate=.001)
metrics = ['accuracy']

model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
model.fit(X_train_padded, y_train, epochs=15, validation_data=(X_val_padded, y_val), verbose=1, 
          callbacks=[checkpoint])

In [ ]:
# model.get_weights()

In [ ]:
history_df = pd.DataFrame(model.history.history)
history_df.head(15)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history_df.loss);

In [ ]:
model.load_weights('../models/bad_words.model/01_checkpoint_0.14.hdf5') # loading weights - model had created erlier
loss, acc = model.evaluate(X_val_padded, y_val)
print(f'Accuracy of restored model {acc*100:.2f}%')

In [ ]:
model = tf.keras.models.load_model('../models/bad_words.model/01_checkpoint_0.14.hdf5')
loss, acc = model.evaluate(X_val_padded, y_val)
print(f'Accuracy of restored model {acc*100:.2f}%')

In [ ]:
!ls '../models/bad_words.model'

In [ ]:
predictions = model.predict(X_train_padded)
predictions = [1 if p > .5 else 0 for p in predictions]

In [ ]:
print(X_train[60:70])
print(y_train[60:70])
print(predictions[60:70])

In [ ]:
train['predictions'] = predictions
train = train[['text_preprocessed', 'labels', 'predictions', 'text']] # columns reorder

In [ ]:
train.head()

In [ ]:
train.to_csv('../datasets/wo_html.csv')

In [ ]:
model.save('../models/modelSequential_wo_HTML.h5')

In [ ]:
#model_new = keras.models.load_model('../models/modelSequential.h5')
model_new = keras.models.load_model('../models/bad_words.model/01_checkpoint_0.14.hdf5')

In [ ]:
predictions = model_new.predict(X_train_padded)
predictions = [1 if p > .5 else 0 for p in predictions]

# Check sentence

In [ ]:
raw_text = "torture"
test_text_preprocessed = bs(raw_text).get_text().replace('\n',' ')
test_text_preprocessed = preprocess(test_text_preprocessed, punctuation_marks, stop_words)
test_text_np = np.array([test_text_preprocessed])
test_sequences = tokenizer.texts_to_sequences(test_text_np)

In [ ]:
print(test_text_preprocessed)
print(test_sequences[0])

In [ ]:
test_sequences_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

In [ ]:
%%time
predictions = model_new.predict(test_sequences_padded)
predictions = [1 if p > .5 else 0 for p in predictions]

In [ ]:
predictions

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# max words in a sequence
max_length = 20

from time import time  # To time our operations
from collections import defaultdict, Counter  # For word frequency

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')

# max words in a sequence
max_length = 20

from time import time  # To time our operations
from collections import defaultdict, Counter  # For word frequency

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')

def preprocess(text, stop_words, punctuation_marks): #, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = token #morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return ' '.join(preprocessed_text)

punctuation_marks = ['!', ',', ';', "'", '(', ')', ':', '-', '--', '', '?', '@', '....', '~',
                     '.', '..', '...', '<', '>', '=', '==', '\'\'', '//', '»', '|', '’', '`', '+',
                     '\"\"', '</', '&', '/', '#', '\'', '*', '``', '%', '[', ']', '{', '}', '$']


stop_words = stopwords.words('english') + ['14000kbps', 'https', "'s", "'m", 'http', 'mp4', 'error', '404']

# loading
with open('../models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    
model = keras.models.load_model('../models/modelSequential.h5')

raw_text = 'nigger'


def predict(sequences):
    sequences_padded = pad_sequences(sequences, maxlen=20, padding="post", truncating="post")
    predictions = model.predict(sequences_padded)
    predictions = [1 if p > .5 else 0 for p in predictions]
    return predictions

# Data loading and preparation
data = pd.read_json('../datasets/neil_ProducerClipSite_rand.json')
mapping = {False: 0, True: 1}
# data.replace({'hasBadWords': mapping}, inplace=True)
# data.hasBadWords = data.hasBadWords.apply(lambda x: 1 if x == True else 0)
# data.rename(columns={"hasBadWords": "labels"}, inplace=True)
data.drop(['SiteID', 'Title', 'Description', 'Keywords', 'Bottom'], axis=1, inplace=True)
data.head()



In [ ]:
data.Top

In [ ]:
%%time
data['text_preprocessed'] = data.apply(lambda row: bs(row.Top, 'lxml').get_text().replace('\n',' '), axis=1)
data['text_preprocessed'] = data.apply(lambda row: preprocess(row.text_preprocessed, punctuation_marks, stop_words), axis=1)

X_val = data.text_preprocessed.to_numpy()
X_val_sequences = tokenizer.texts_to_sequences(X_val)
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_length, padding="post", truncating="post")

predictions = model.predict(X_val_padded)
predictions = [1 if p > .5 else 0 for p in predictions]


In [ ]:
data['predictions'] = predictions

In [ ]:
data.head()

In [ ]:
data = data[['Top', 'predictions']]

In [ ]:
data.to_csv('../datasets/last_one.csv')

In [ ]:
data.to_json('../datasets/last_one.json')